In [78]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from costcla.datasets import load_creditscoring1
from costcla.models import CostSensitiveRandomForestClassifier
from costcla.metrics import cost_loss
from sklearn.metrics import accuracy_score

import pandas as pd
import numpy as np

In [84]:
X = pd.read_csv("data/X.csv", index_col=0)
y = pd.read_csv("data/y.csv", index_col=0)
cost_matrix = np.stack(y["class"].apply(lambda i: np.array([5, 1, 0, 0]) if i == 0 else np.array([1, 5, 0, 0])))

In [115]:
X_train, X_test, y_train, y_test, mat_train, mat_test = train_test_split(X, y["class"], cost_matrix, test_size=.15, random_state=16)

In [119]:
for combination in (["majority_voting", "weighted_voting", "stacking", "stacking_proba", "stacking_bmr", "stacking_proba_bmr", "majority_bmr", "weighted_bmr"]):
    try: 
        f = CostSensitiveRandomForestClassifier(n_estimators = 100, combination = combination).fit(X_train.values, y_train.values, mat_train)
        pred = f.predict(X_test.values)
        print(combination,": acc=",accuracy_score(y_test.values, pred),", saving=",savings_score(y_test.values, pred, mat_test))
    except Exception:
        pass

majority_voting : acc= 0.7 , saving= 0.0
weighted_voting : acc= 0.7066666666666667 , saving= 0.022222222222222254
stacking : acc= 0.26666666666666666 , saving= -1.4444444444444446
stacking_proba : acc= 0.31333333333333335 , saving= -1.2888888888888888


In [177]:
pred_noncost = RandomForestClassifier(n_jobs=-1).fit(X_train.values, y_train.values).predict(X_test.values)
print(combination,": acc=",accuracy_score(y_test.values, pred_noncost),", saving=",savings_score(y_test.values, pred_noncost, mat_test))

weighted_bmr : acc= 0.78 , saving= 0.2666666666666667
